In [1]:
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import pims

from sklearn import mixture
from sklearn.decomposition import PCA
import time
import sys
import scipy.ndimage
import scipy.ndimage as ndimage
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
from matplotlib.colors import LogNorm
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import GridSearchCV

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
#%matplotlib notebook
import seaborn as sns
%config InlineBackend.figure_format = 'retina'
from pylab import rcParams
rcParams['figure.figsize'] = (9, 6)
import warnings
warnings.filterwarnings(action='once')

In [2]:
seg_len = 15

In [3]:
def classify1(features, MX=5):
    newdata = features
    n_components = np.arange(1, MX + 1)
    BIC = np.zeros(n_components.shape)
    lowest_bic = np.infty
    best_gmm = None

    for i, n in enumerate(n_components):        
        gmm = GaussianMixture(n_components=n, covariance_type='full', random_state=0)
        gmm.fit(newdata)
        BIC[i] = gmm.bic(newdata)
        print(BIC[i])
        if BIC[i] < lowest_bic:
            lowest_bic = BIC[i]
            best_gmm = gmm

    best_gmm = GaussianMixture(n_components=best_gmm.n_components, covariance_type='full', random_state=0)
    best_gmm.fit(features)

    print(best_gmm.n_components)
    plt.figure()
    plt.plot(BIC)
    plt.show()
    return best_gmm

def classify2(features, MX, MN=1):
    parameters = {
        'n_components' : np.arange(MN, MX + 1)
    }
    clf = GridSearchCV(GaussianMixture(covariance_type='full', random_state=0), parameters, cv=5, n_jobs=-1)
    clf.fit(features)
    print("n_components", clf.best_estimator_.n_components)
    return clf.best_estimator_

def plot_array(arr):
    ny, nx = arr.shape
    xv, yv = np.meshgrid(np.linspace(0, nx, nx), np.linspace(0, ny, ny))
    fig = plt.figure()
    ax = Axes3D(fig)
    ax.plot_surface(xv,yv,arr)
    plt.show()

In [4]:
df = pd.read_csv("data.csv")
df.drop(df.columns[0], inplace=True, axis=1)
scaler = MinMaxScaler()
df2 = df.drop([df.columns[0], df.columns[1]], axis=1)
df2 = pd.DataFrame(scaler.fit_transform(df2), columns=df.columns[2:])
display(df2.describe())
display(df2.head(5))

df_test = pd.read_csv("data.csv")
df_test.drop(df_test.columns[0], inplace=True, axis=1)
df2_test = df_test.drop([df_test.columns[0], df_test.columns[1]], axis=1)
df2_test = pd.DataFrame(scaler.transform(df2_test), columns=df_test.columns[2:])

,x,y,w,h,Rf,mx,my
count,2058.000000,2058.000000,2058.000000,2058.000000,2058,2058.000000,2058.000000
mean,0.573969,0.391572,0.181708,0.180628,0,0.791784,0.535095
std,0.186654,0.231454,0.132961,0.143815,0,0.122342,0.147062
min,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000
25%,0.453431,0.201754,0.089286,0.075000,0,0.754386,0.458333
50%,0.540441,0.320175,0.142857,0.133333,0,0.824561,0.550000
75%,0.721814,0.523575,0.250000,0.250000,0,0.877193,0.633333
max,1.000000,1.000000,1.000000,1.000000,0,1.000000,1.000000


,x,y,w,h,Rf,mx,my
0,0.453431,0.096491,0.035714,0.016667,0,0.912281,0.533333
1,0.496324,0.131579,0.026786,0.033333,0,0.824561,0.591667
2,0.563725,0.109649,0.053571,0.050000,0,0.877193,0.600000
3,0.627451,0.122807,0.053571,0.066667,0,0.912281,0.508333
4,0.692402,0.201754,0.098214,0.083333,0,0.859649,0.483333


In [5]:
bgmm = classify2(df2, 5)

n_components 5


In [6]:
df3 = pd.DataFrame(bgmm.predict_proba(df2))
df3["time"] = df["time"]
df3["frame"] = df["frame"]
df3 = df3.groupby(['time', 'frame']).agg('max')
df3.reset_index(inplace=True)

df3_test = pd.DataFrame(bgmm.predict_proba(df2_test))
df3_test["time"] = df_test["time"]
df3_test["frame"] = df_test["frame"]
df3_test = df3_test.groupby(['time', 'frame']).agg('max')
df3_test.reset_index(inplace=True)

#display(df3.head(11))

df3 = df3.apply(lambda x: [x['time'], x['frame']] + list(x[2:] / x[2:].sum()), axis=1)
df3_test = df3_test.apply(lambda x: [x['time'], x['frame']] + list(x[2:] / x[2:].sum()), axis=1)

display(df3.head(11))

#arr = df3_test.drop(["frame", "time"], axis=1)
#display(np.array(arr))
#display(np.array(arr) / np.linalg.norm(arr, axis=1))

PN = []

last_t = df3.iloc[0]["time"]

pn = []

i = 0
i_next = 0
i_next_tm = 0
while i < len(df3):
    r = df3.iloc[i]
    pn.append(r.drop(["time", "frame"]).ravel())
    if (i_next == 0) and (r.time - last_t > int(seg_len / 2.0)):
        i_next = i
        i_next_tm = r.time
    if r.time - last_t > seg_len:
        last_t = i_next_tm
        i = i_next
        i_next = 0
        PN.append(pn)
        pn = []
    else:
        i += 1

#display(PN[0])        
        
print("N =",len(PN))
for i in range(len(PN)):
    print("N[",i,"]=",len(PN[i]))

,time,frame,0,1,2,3,4
0,2.08,52,0.000566,0.331237,0.331749,0.331732,0.004716
1,3.04,76,0.000575,0.489115,0.014039,0.489418,0.006853
2,4.12,103,0.000998,0.328769,0.329795,0.329745,0.010693
3,5.08,127,0.000435,0.328952,0.011977,0.329423,0.329214
4,6.04,151,0.250847,0.250138,0.245194,0.251102,0.002720
5,7.12,178,0.000288,0.359483,0.276337,0.360067,0.003826
6,8.08,202,0.017559,0.441591,0.006782,0.443243,0.090824
7,9.04,226,0.003028,0.332269,0.000555,0.333719,0.330429
8,10.12,253,0.002383,0.332784,0.000271,0.333449,0.331113
9,11.08,277,0.004972,0.331549,0.000023,0.331709,0.331747


N = 29
N[ 0 ]= 17
N[ 1 ]= 16
N[ 2 ]= 17
N[ 3 ]= 17
N[ 4 ]= 17
N[ 5 ]= 17
N[ 6 ]= 17
N[ 7 ]= 16
N[ 8 ]= 16
N[ 9 ]= 17
N[ 10 ]= 17
N[ 11 ]= 17
N[ 12 ]= 17
N[ 13 ]= 17
N[ 14 ]= 17
N[ 15 ]= 17
N[ 16 ]= 17
N[ 17 ]= 17
N[ 18 ]= 17
N[ 19 ]= 17
N[ 20 ]= 17
N[ 21 ]= 17
N[ 22 ]= 17
N[ 23 ]= 17
N[ 24 ]= 17
N[ 25 ]= 17
N[ 26 ]= 17
N[ 27 ]= 17
N[ 28 ]= 17


In [7]:
from hmmlearn import hmm

def my_hmm_score(h, samples):
    s = h.score(samples)
    return 0 if s > 0 else s

hmms1 = []

N = len(PN)

for i in range(N):    
    hmm1 = hmm.GaussianHMM(n_components=bgmm.n_components, covariance_type="full")    
    hmm1.fit(PN[i])
    if i == 0:
        display(hmm1.means_)
        display(hmm1.covars_)
    hmms1.append(hmm1)

/usr/local/lib/python3.5/dist-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/hmmlearn-0.2.1-py3.5-linux-x86_64.egg/hmmlearn/base.py:465: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  return logsumexp(fwdlattice[-1]), fwdlattice
/usr/local/lib/python3.5/dist-packages/hmmlearn-0.2.1-py3.5-linux-x86_64.egg/hmmlearn/utils.py:46: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  a_lse = logsumexp(a, axis)


array([[  5.25168351e-04,   4.58169540e-01,   4.58309635e-01,
          4.83285481e-16,   8.29956568e-02],
       [  3.53139138e-02,   3.01916294e-01,   5.19258964e-02,
          3.07219895e-01,   3.03624000e-01],
       [  6.31746329e-02,   3.17406631e-01,   2.95768848e-01,
          3.18161387e-01,   5.48850039e-03],
       [  4.52319261e-03,   3.52582271e-05,   9.28012300e-04,
          4.96957991e-01,   4.97555546e-01],
       [  6.19006490e-03,   4.19885990e-01,   1.09327608e-02,
          4.20694410e-01,   1.42296774e-01]])

array([[[ 0.01      ,  0.01      ,  0.01      ,  0.01      ,  0.01      ],
        [ 0.01      ,  0.01      ,  0.01      ,  0.01      ,  0.01      ],
        [ 0.01      ,  0.01      ,  0.01      ,  0.01      ,  0.01      ],
        [ 0.01      ,  0.01      ,  0.01      ,  0.01      ,  0.01      ],
        [ 0.01      ,  0.01      ,  0.01      ,  0.01      ,  0.01      ]],

       [[ 0.00865073, -0.0013024 , -0.00016063, -0.00019333, -0.00074438],
        [-0.0013024 ,  0.00299735, -0.00087781,  0.00262542,  0.00280744],
        [-0.00016063, -0.00087781,  0.00940196, -0.00112124, -0.00099228],
        [-0.00019333,  0.00262542, -0.00112124,  0.00242017,  0.00251897],
        [-0.00074438,  0.00280744, -0.00099228,  0.00251897,  0.00266025]],

       [[ 0.01424033, -0.0017107 , -0.00065948, -0.0016976 ,  0.00232744],
        [-0.0017107 ,  0.00415395,  0.00336719,  0.00414838,  0.00254118],
        [-0.00065948,  0.00336719,  0.00384697,  0.00336492,  0.0025804 ],
        [-0.0016976 ,

In [8]:
Sij = np.zeros((N,N), np.float64)

for i in range(N):
    for j in range(N):
        print(i, j)
        s1 = my_hmm_score(hmms1[i], PN[j]) * (1.0 / len(PN[j]))
        s2 = my_hmm_score(hmms1[j], PN[i]) * (1.0 / len(PN[i]))
        Sij[i, j] = 0.5 * (s1 + s2)

0 0


ValueError: rows of transmat_ must sum to 1.0 (got [ 1.  1.  1.  0.  1.])

In [ ]:
from scipy.linalg import fractional_matrix_power

Lij = np.zeros((N,N), np.float64)

for i in range(N):
    s1 = 0
    for j in range(N):
        s1 += Sij[i, j]
    Lij[i, i] = s1
Lij2 = fractional_matrix_power(Lij, -0.5)
Snorm = np.dot(np.dot(Lij2, Sij), Lij2).astype('float64')
plot_array(Snorm)

In [ ]:
plt.figure()
plt.imshow(Snorm, cmap='gray')
plt.show()

In [ ]:
Km = int(N / 5)
Snorm_vals, Snorm_vecs = np.linalg.eig(Snorm)
indices = np.argsort(Snorm_vals)[::-1][:Km]
top_vecs = Snorm_vecs[indices].T
display(top_vecs)
print(Km)
eig_gmm = classify2(top_vecs, Km, Km)
finalK = eig_gmm.n_components

In [ ]:
final_hmms = []
final_nk = []
top_classes = eig_gmm.predict(top_vecs)
for i in range(finalK):    
    tmp1 = []
    num = 0
    tmp1_ls = []
    for j in range(N):
        if top_classes[j] == i:
            tmp1 += PN[j]
            tmp1_ls.append(len(PN[j]))
            num += 1
    hmm1 = hmm.GaussianHMM(n_components=bgmm.n_components)
    #hmm1.fit(tmp1, tmp1_ls)
    hmm1.fit(tmp1)
    final_hmms.append(hmm1)
    final_nk.append(num)    

In [ ]:
def calc_final_prob(samples):
    res = 0
    for i in range(finalK):                                
        res += np.exp(my_hmm_score(final_hmms[i], samples)) * (float(final_nk[i]) / float(N))        
    return res

TestPN = df3_test.drop(["time", "frame"], axis=1)
test_arr = []
scrs = []
scrs_prob = []
j = 1
for i in range(len(TestPN)):
    r = TestPN.iloc[i].ravel()
    test_arr.append(r)
    prb = calc_final_prob(test_arr)    
    prb2 = np.log(prb)
    if prb2 == -np.inf:
        prb2 = -1000;    
    scrs.append(prb2 / j)
    scrs_prob.append(prb)
    j += 1
    if j >= 17000:
        j -= 1
        del test_arr[0]        
        
plt.figure()
plt.ylim([-0.1, 1.1])
plt.plot(scrs_prob)
plt.show()

In [ ]:
import traceback

Qt = 0
alpha = 0.5

first_frm = 0
first_frm_num = 0

qts = {}
qts_lst = []
for i in range(len(scrs)):
    if i == 0:
        Qt = scrs[0]
    else:
        Qt = (1 - alpha) * Qt + alpha * (scrs[i] - scrs[i - 1])        
    qts[df3_test.iloc[i]["frame"]] = Qt
    qts_lst.append(Qt)
    if i == first_frm:
        first_frm_num = df3_test.iloc[i]["frame"]
            
plt.figure()
plt.plot(qts_lst[first_frm:])
plt.show()

cap = cv2.VideoCapture('Datasets/UCSDPed1/combined/test.avi')
#cap = cv2.VideoCapture('z3.avi')
ok, frame = cap.read()
aspect = float(frame.shape[1]) / frame.shape[0]
cap.release()
cap = cv2.VideoCapture('Datasets/UCSDPed1/combined/test.avi')
#cap = cv2.VideoCapture('z3.avi')

try:
    while True:
        ok, frame = cap.read()
        if not ok:
            break
        frame = cv2.resize(frame, (int(240 * aspect), 240), interpolation = cv2.INTER_AREA)    
        fnum = cap.get(cv2.CAP_PROP_POS_FRAMES)    
        delay = 1
        if (fnum >= first_frm_num) and (fnum in qts):
            if qts[fnum] < -0.015:
                cv2.rectangle(frame, (0,0), (frame.shape[1], frame.shape[0]), (0,0,255), 10)
                delay = 0
        cv2.imshow('frame', frame)
        k = cv2.waitKey(delay) & 0xff
        if k == 32:
            k = cv2.waitKey() & 0xff
        if k == 27:
            break
except Exception as e:    
    traceback.print_exc()

cap.release()
cv2.destroyAllWindows()